In [1]:
import requests
import numpy as np
import json
import time
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

np.set_printoptions(precision=5)

In [2]:
NAMESPACE = 'kfserving-test'
MODEL_NAME =  'product-recommender' #'autoencoder-recommender' 'product-recommender' 'concept-drift'
KFSERVING_VERSION = 'v1beta1'

## Load preprocessed Data from Training of the Recommender Model

In [3]:
sessions_padded = np.load('list_sessions_padded.npy')
print(sessions_padded.shape)
last_clicked = np.load('list_last_clicked.npy')
print(last_clicked.shape)
n_output_features = int(last_clicked.max())
n_unique_input_ids = int(sessions_padded[:,:,0].max())
window_length = sessions_padded.shape[1]
n_input_features = sessions_padded.shape[2]

id_mapping = pd.read_csv('ID_Mapping.csv')
id_mapping

(45916, 207, 52)
(45916,)


,Unnamed: 0,Item_ID,Mapped_ID,category_code
0,0,2402273,1,appliances.personal.massager
1,1,20100164,2,apparel.trousers
2,2,21400264,3,electronics.clocks
3,3,1005239,4,construction.tools.light
4,4,5100885,5,computers.notebook
...,...,...,...,...
38510,38510,6902925,38511,electronics.telephone
38511,38511,2602249,38512,appliances.kitchen.refrigerators
38512,38512,20100099,38513,apparel.trousers
38513,38513,21405267,38514,electronics.clocks


## Request Recommender Model

In [63]:
def request_kf_serving(np_array, MODEL_NAME, NAMESPACE, INGRESS_HOST, INGRESS_PORT, item_ground_truth):
    data = json.dumps({"instances": np_array.tolist(),
                      'id': item_ground_truth})
    headers = {#"content-type": "application/json",
               'Host': '{}.{}.example.com'.format(MODEL_NAME, NAMESPACE)}
    json_response = requests.post(
        'http://{}:{}/v1/models/{}:predict'.format(INGRESS_HOST, INGRESS_PORT, MODEL_NAME), data=data, headers=headers)

    try:
        predictions = json.loads(json_response.text)['predictions']
    except Exception as e:
        raise e
    return np.array(predictions).astype(np.float32)



INGRESS_HOST_LIST = !kubectl get po -l istio=ingressgateway -n istio-system -o jsonpath={.items[0].status.hostIP}
INGRESS_HOST =  INGRESS_HOST_LIST[0] #eg. '192.168.52.86'
#print(INGRESS_HOST)
INGRESS_PORT = 31017

example = 14 #5,6,7, 13
start = time.time()

pred = request_kf_serving(sessions_padded[example:example+1],#[np.newaxis,:,:],
                          MODEL_NAME, NAMESPACE, INGRESS_HOST, INGRESS_PORT, int(last_clicked[example]))   
end = time.time()
print(end - start)

top = pred.argsort()[0][::-1][:5]
print("Session:")
session = pd.DataFrame()
session['category_code'] = [id_mapping['category_code'][int(i)-1] for i in sessions_padded[example,:,0] if i>0]
session['Item_ID'] = [id_mapping['Item_ID'][int(i)-1] for i in sessions_padded[example,:,0] if i>0]
session['Item_ID_Mapped'] = [int(i) for i in sessions_padded[example,:,0] if i>0]
session


print("Prediction:")
prediction = pd.DataFrame()
prediction['category_code'] = [id_mapping['category_code'][int(i)-1] for i in top if i>0]
prediction['Item_ID'] = [id_mapping['Item_ID'][int(i)-1] for i in top if i>0]
prediction['Item_ID_Mapped'] = [int(i) for i in top if i>0]
prediction['probability'] = pred[0, top]
prediction
print("Ground Truth:", last_clicked[example])

0.1285841464996338
Session:


,category_code,Item_ID,Item_ID_Mapped
0,kids.skates,23301564,10804
1,construction.tools.generator,15901606,2963
2,furniture.kitchen.table,31300060,24062
3,apparel.shoes.keds,100011061,559
4,apparel.shoes.keds,16400390,2757
5,apparel.shoes.keds,100011061,559
6,auto.accessories.winch,24400437,24615


Prediction:


,category_code,Item_ID,Item_ID_Mapped,probability
0,apparel.shoes,5300300,24969,0.999923
1,appliances.kitchen.kettle,20000602,17781,0.000022
2,electronics.clocks,5100266,14029,0.000006
3,electronics.clocks,100005669,1928,0.000005
4,electronics.video.tv,1802075,21568,0.000004


Ground Truth: 24969


## Test autoscaling

In [ ]:
import threading

while True:
    for _ in range(10):
        x = threading.Thread(target=request_kf_serving, args=(np.ones((1, 207, 52), dtype=np.float32),
                               MODEL_NAME, NAMESPACE, INGRESS_HOST, INGRESS_PORT))
        x.start()
    time.sleep(0.3)